# Assignment 4

## 1.Using the same Dmart file, calculate the weekly returns for the entire year 

2.Populate a table of returns with week number of month(1,2,3,4,5) and month name        

Please note:
1.Week starts from Monday
2. Use while loop to loop through the dataframe and not any form of groupby, directly or indirectly
3. You buy at the open of the first day of the week and sell at the close of the last day of week
4. If the end of month falls on Wednesday, you exit your position on Wednesday close. New week of the new month starts from Thursday


In [56]:
import pandas as pd
import numpy as np
import calendar
import datetime
from dateutil import parser

import warnings
warnings.filterwarnings('ignore')

In [57]:
#Please change the path of the file as per your pc after r'
path = r'C:\Users\admin\Desktop\Sakshi\6. Python\Assignment 4\Dmart.csv'

##### Dmart file = df

In [58]:
df = pd.read_csv(path)

In [59]:
#converted date/time column to date/time format

df['Date'] = np.nan

for i in range (0,len(df['Date/Time'])):
    date_string = df['Date/Time'][i]
    date_object = parser.parse(date_string)
    df['Date'][i] = date_object

In [60]:
#Adding a day column, month column, week number column
df['Day'] = np.nan
df['Month'] = np.nan
df['Week Number'] = np.nan

 
for i in range (0,len(df['Date/Time'])):
    df['Day'][i] = (df['Date'][i]).strftime("%A")
    df['Month'][i] = (df['Date'][i]).strftime("%m")
    df['Week Number'][i] = (df['Date'][i]).strftime("%W")  


In [61]:
#adding a column to check if day is wednesday or no

df['Checking day is wednesday or no'] = df['Day'].apply (lambda x: '1' if x == 'Wednesday' else '0')

In [62]:
#adjusting month to another cell
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if i == 251:
            df.loc[i,'NewLocation'] = df.loc[i,'Month']
        else:
            df.loc[i,'NewLocation'] = df.loc[i+1,'Month']
        i += 1

In [63]:
#finding month end
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if df.loc[i,'Month'] == df.loc[i,'NewLocation']:
            df.loc[i,'MonthCheck'] = '0'
        else:
            df.loc[i,'MonthCheck'] = '1'
        i += 1

In [64]:
# combining month end and wednesday close
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if ((df.loc[i,'MonthCheck'] == '1') and (df.loc[i,'Checking day is wednesday or no'] == '1')):
            df.loc[i,'WednesdayClose'] = '1'
        else:
            df.loc[i,'WednesdayClose'] = '0'
        i += 1    

In [66]:
#close counter helper
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'CloseApprox'] = '0'
        elif i == 251:
            df.loc[i,'CloseApprox'] = '1'
        elif (df.loc[i,'Week Number']) == (df.loc[i+1,'Week Number']):
            df.loc[i,'CloseApprox'] = '0'
        else:
            df.loc[i,'CloseApprox'] = '1'
        i += 1    

In [67]:
#to determine the close
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if df.loc[i,'WednesdayClose'] == '1':
            df.loc[i,'CloseCount'] = 1
        elif df.loc[i,'CloseApprox'] == '1':
            df.loc[i,'CloseCount'] = 1
        else:
            df.loc[i,'CloseCount'] = 0
        i += 1

In [68]:
#to determine the open count
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'OpenCount'] = 1
        elif i == 251:
            df.loc[i,'OpenCount'] = 0
        elif df.loc[i,'Week Number'] != df.loc[i-1,'Week Number']:
            df.loc[i,'OpenCount'] = 1
        else:
            df.loc[i,'OpenCount'] = 0
        i += 1

In [70]:
#counters of close and open 
i = 0
summationclose = 0
for i in range(0,len(df['Date/Time'])):
    summationclose += df.loc[i,'CloseCount']
    df.loc[i,'CloseCounter'] = summationclose
    
j = 0
summationopen = 0
for j in range (0,len(df['Date/Time'])):
    summationopen += df.loc[j,'OpenCount']
    df.loc[j,'OpenCounter'] = summationopen

In [71]:
#close counter helper
i = 0
while i < len(df):
    for i in range(0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'CloseCounterHelper'] = df.loc[i,'CloseCounter'] 
        elif df.loc[i,'CloseCounter'] != df.loc[i,'OpenCounter']:
            df.loc[i,'CloseCounterHelper'] = df.loc[i-1,'CloseCounter']
        else:
            df.loc[i,'CloseCounterHelper'] = df.loc[i,'CloseCounter']
        i+=1

In [72]:
#creating the final open count
i = 0
while i < len(df):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'OpenCounterFinal'] = df.loc[i,'OpenCounter']
        elif df.loc[i,'OpenCounter'] != df.loc[i-1,'OpenCounter']:
            df.loc[i,'OpenCounterFinal'] = df.loc[i,'OpenCounter']
        else:
            df.loc[i,'OpenCounterFinal'] = 0
        i+= 1

In [73]:
#reaching a step closer to close count final
i = 0
while i < len(df):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
        elif i == 251:
            df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
        elif (df.loc[i,'CloseCounterHelper'] == df.loc[i,'OpenCounterFinal']):
            df.loc[i,'CloseCounterFinal'] = 0
        elif (df.loc[i,'CloseCounterHelper'] != df.loc[i-1,'CloseCounterHelper']):
            df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
        elif (df.loc[i,'CloseCounterHelper'] != df.loc[i+1,'CloseCounterHelper']):
            df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
        else:
            df.loc[i,'CloseCounterFinal'] = 0
        i += 1

In [74]:
#the final close count
i = 0
while i < len(df):
    for i in range (0,len(df['Date/Time'])):
        if i == 251:
            df.loc[i,'CloseFinal1'] = df.loc[i,'CloseCounterFinal']
        elif df.loc[i+1,'CloseCounterFinal'] != 0:
            df.loc[i,'CloseFinal1'] = 0
        else:
            df.loc[i,'CloseFinal1'] = df.loc[i,'CloseCounterFinal']
        i += 1

### thus the main open counter column = 'OpenCounterFinal' and for close it is 'CloseFinal1' 

In [75]:
i = 0
while i < len(df):
    for i in range (0,len(df['Date/Time'])):
        if df.loc[i,'OpenCounterFinal'] != 0:
            df.loc[i,'BuyPrice'] = df.loc[i,'Open']
        else:
            df.loc[i,'BuyPrice'] = 0
        i += 1

j = 0
while j < len(df):
    for j in range (0,len(df['Date/Time'])):
        if df.loc[j,'CloseFinal1'] != 0:
            df.loc[j,'SellPrice'] = df.loc[j,'Close']
        else:
            df.loc[j,'SellPrice'] = 0
        j += 1

In [76]:
BuyData = pd.DataFrame({
    'Buy':[],
    'Month': []})

i = 0
while i < len(df):
    for i in range (0,len(df)):
        if df.loc[i,'BuyPrice'] != 0:
            BuyData.loc[i,'Buy'] = df.loc[i,'BuyPrice']
            BuyData.loc[i,'Month'] = df.loc[i,'Month']
            
        i+=1
        
BuyData.reset_index(inplace = True,drop = True)           

In [77]:
SellData = pd.DataFrame({
    'Sell':[]})

i = 0
while i < len(df):
    for i in range (0,len(df)):
        if df.loc[i,'SellPrice'] != 0:
            SellData.loc[i,'Sell'] = df.loc[i,'SellPrice']
        i+=1
        
SellData.reset_index(inplace = True,drop = True)            

In [78]:
Returns = pd.merge(BuyData,SellData,left_index = True,right_index = True)

In [79]:
Returns['ROC'] = ((Returns['Sell']/Returns['Buy'])-1)*100

In [81]:
Table1 = pd.DataFrame({'January':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '01':
            Table1.loc[i,'January'] = Returns.loc[i,'ROC']
        i+=1   

Table1.reset_index(inplace = True,drop = True)

In [82]:
Table2 = pd.DataFrame({'February':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '02':
            Table2.loc[i,'February'] = Returns.loc[i,'ROC']
        i+=1   

Table2.reset_index(inplace = True,drop = True)

In [83]:
Table3 = pd.DataFrame({'March':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '03':
            Table3.loc[i,'March'] = Returns.loc[i,'ROC']
        i+=1   

Table3.reset_index(inplace = True,drop = True)

In [84]:
Table4 = pd.DataFrame({'April':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '04':
            Table4.loc[i,'April'] = Returns.loc[i,'ROC']
        i+=1   

Table4.reset_index(inplace = True,drop = True)

In [85]:
Table5 = pd.DataFrame({'May':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '05':
            Table5.loc[i,'May'] = Returns.loc[i,'ROC']
        i+=1   

Table5.reset_index(inplace = True,drop = True)

In [86]:
Table6 = pd.DataFrame({'June':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '06':
            Table6.loc[i,'June'] = Returns.loc[i,'ROC']
        i+=1   

Table6.reset_index(inplace = True,drop = True)

In [87]:
Table7 = pd.DataFrame({'July':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '07':
            Table7.loc[i,'July'] = Returns.loc[i,'ROC']
        i+=1   

Table7.reset_index(inplace = True,drop = True)

In [88]:
Table8 = pd.DataFrame({'August':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '08':
            Table8.loc[i,'August'] = Returns.loc[i,'ROC']
        i+=1   

Table8.reset_index(inplace = True,drop = True)

In [89]:
Table9 = pd.DataFrame({'September':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '09':
            Table9.loc[i,'September'] = Returns.loc[i,'ROC']
        i+=1   

Table9.reset_index(inplace = True,drop = True)

In [90]:
Table10 = pd.DataFrame({'October':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '10':
            Table10.loc[i,'October'] = Returns.loc[i,'ROC']
        i+=1   

Table10.reset_index(inplace = True,drop = True)

In [91]:
Table11 = pd.DataFrame({'November':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '11':
            Table11.loc[i,'November'] = Returns.loc[i,'ROC']
        i+=1   

Table11.reset_index(inplace = True,drop = True)

In [92]:
Table12 = pd.DataFrame({'December':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '12':
            Table12.loc[i,'December'] = Returns.loc[i,'ROC']
        i+=1   

Table12.reset_index(inplace = True,drop = True)

In [93]:
H0 = pd.DataFrame({'Weeks':['Week1','Week2','Week3','Week4','Week5']})

In [94]:
Output = H0.join([Table1,Table2,Table3,Table4,Table5,Table6,Table7,Table8,Table9,Table10,Table11,Table12])
display(Output)

,Weeks,January,February,March,April,May,June,July,August,September,October,November,December
0,Week1,-0.332486,9.750480,-3.976445,14.475384,-2.389404,7.710875,1.162892,3.369266,-1.350220,-6.172051,2.437500,4.802575
1,Week2,2.733990,5.296870,-3.844394,-4.650862,2.359649,-4.849206,-9.968100,1.851852,-4.486667,-4.237452,2.953102,9.959458
2,Week3,3.405195,4.593332,-4.000601,3.612189,2.798971,-0.415086,2.601141,3.596974,-3.324138,7.490922,0.355917,-2.871087
3,Week4,-2.570000,-5.098416,14.250398,3.636957,-2.977768,-3.026205,0.951566,2.178462,3.500876,5.655660,-3.045191,1.062382
4,Week5,6.504755,NaN,-0.613069,NaN,NaN,-0.185234,NaN,-2.064031,NaN,NaN,NaN,2.890970


## Presenting all the displays below

In [95]:
display(df)

,Ticker,Date/Time,Open,Low,High,Close,Date,Day,Month,Week Number,...,CloseCount,OpenCount,CloseCounter,OpenCounter,CloseCounterHelper,OpenCounterFinal,CloseCounterFinal,CloseFinal1,BuyPrice,SellPrice
0,DMART.EQ-NSE,1/1/20,1849.70,1820.00,1867.00,1828.50,2020-01-01 00:00:00,Wednesday,01,00,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1849.70,0.00
1,DMART.EQ-NSE,1/2/20,1831.00,1815.00,1854.85,1824.20,2020-01-02 00:00:00,Thursday,01,00,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00,0.00
2,DMART.EQ-NSE,1/3/20,1827.00,1827.00,1853.00,1843.55,2020-01-03 00:00:00,Friday,01,00,...,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.00,1843.55
3,DMART.EQ-NSE,1/6/20,1827.00,1780.00,1829.00,1789.90,2020-01-06 00:00:00,Monday,01,01,...,0.0,1.0,1.0,2.0,1.0,2.0,0.0,0.0,1827.00,0.00
4,DMART.EQ-NSE,1/7/20,1805.00,1805.00,1837.50,1815.75,2020-01-07 00:00:00,Tuesday,01,01,...,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,DMART.EQ-NSE,12/24/2020,2661.85,2604.80,2700.00,2673.10,2020-12-24 00:00:00,Thursday,12,51,...,1.0,0.0,53.0,52.0,52.0,0.0,52.0,52.0,0.00,2673.10
248,DMART.EQ-NSE,12/28/2020,2685.95,2655.00,2698.95,2678.00,2020-12-28 00:00:00,Monday,12,52,...,0.0,1.0,53.0,53.0,53.0,53.0,0.0,0.0,2685.95,0.00
249,DMART.EQ-NSE,12/29/2020,2690.00,2645.00,2694.70,2665.70,2020-12-29 00:00:00,Tuesday,12,52,...,0.0,0.0,53.0,53.0,53.0,0.0,0.0,0.0,0.00,0.00
250,DMART.EQ-NSE,12/30/2020,2666.20,2642.35,2725.00,2680.30,2020-12-30 00:00:00,Wednesday,12,52,...,0.0,0.0,53.0,53.0,53.0,0.0,0.0,0.0,0.00,0.00


In [96]:
display(Returns)

,Buy,Month,Sell,ROC
0,1849.70,01,1843.55,-0.332486
1,1827.00,01,1876.95,2.733990
2,1925.00,01,1990.55,3.405195
3,2000.00,01,1948.60,-2.570000
4,1945.50,01,2072.05,6.504755
5,2084.00,02,2287.20,9.750480
6,2287.20,02,2408.35,5.296870
7,2354.50,02,2462.65,4.593332
8,2448.80,02,2323.95,-5.098416
9,2335.00,03,2242.15,-3.976445


In [97]:
display(Output)

,Weeks,January,February,March,April,May,June,July,August,September,October,November,December
0,Week1,-0.332486,9.750480,-3.976445,14.475384,-2.389404,7.710875,1.162892,3.369266,-1.350220,-6.172051,2.437500,4.802575
1,Week2,2.733990,5.296870,-3.844394,-4.650862,2.359649,-4.849206,-9.968100,1.851852,-4.486667,-4.237452,2.953102,9.959458
2,Week3,3.405195,4.593332,-4.000601,3.612189,2.798971,-0.415086,2.601141,3.596974,-3.324138,7.490922,0.355917,-2.871087
3,Week4,-2.570000,-5.098416,14.250398,3.636957,-2.977768,-3.026205,0.951566,2.178462,3.500876,5.655660,-3.045191,1.062382
4,Week5,6.504755,NaN,-0.613069,NaN,NaN,-0.185234,NaN,-2.064031,NaN,NaN,NaN,2.890970


In [99]:
df.to_csv('DmartOutput.csv')
Returns.to_csv('ReturnsOutput.csv')
Output.to_csv('FinalRequiredOutput.csv')